In [1]:
import datetime
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import time
from bs4 import BeautifulSoup
import requests
import re
from datetime import datetime
import string
import pytz
from matplotlib import pyplot
import matplotlib.cm as cm

In [2]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)

# fetch company names and price

In [19]:
def get_nyse_stocks():
    
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.13; rv:63.0) Gecko/20100101 Firefox/63.0'}

    list_stock = []
    for letter in string.ascii_uppercase:
        print("Fetching for letter: " + letter)
        
        r = requests.get("https://eoddata.com/stocklist/NYSE/" + letter + ".htm", headers=headers)
        html_doc = r.text
        soup = BeautifulSoup(html_doc, 'html.parser')
        soup_tag = soup.find_all("a", href=True)

        for link in soup_tag:
            text = link.get('href')
            if "/stockquote/NYSE/" in text:
                text = text.split("/")[-1].split(".")[0]

                if text not in list_stock:
                    list_stock.append(text)
                    
    return list_stock

In [213]:
def get_interesting_stocks_wsj(list_stock):
    
    interesting_stocks = []
    
    for symbol in list_stock:
        
        print(symbol)

        headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.13; rv:63.0) Gecko/20100101 Firefox/63.0'}
        r = requests.get("https://www.wsj.com/market-data/quotes/" + symbol + "/research-ratings", headers=headers)
        html_doc = r.text
        soup = BeautifulSoup(html_doc, 'html.parser')
        soup_tag = soup.find_all("span", attrs={"class": "data_data"})

        def get_prices(soup_tag):
            
            current_price_tag = re.findall(r'\d+', str(soup_tag[-1]))
            low_price_tag = re.findall(r'\d+', str(soup_tag[-3]))
            
            def convert_tag_to_price(tag_price):
                
                # price of the action is in thousands (seperated by comma)
                if len(tag_price) == 3:
                    price = int(tag_price[0])*1000+int(tag_price[1])+float(tag_price[2])/100
                else:
                    price = int(tag_price[0])+float(tag_price[1])/100

                return price
            
            current_price = convert_tag_to_price(current_price_tag)
            low_price = convert_tag_to_price(low_price_tag)
            
            return current_price, low_price
        
        def get_number_of_analysts(soup_tag):
            
            analyst_list = [int(re.findall(r'\d+', str(tag))[0]) for tag in soup_tag if len(re.findall(r'\d+', str(tag))) == 1]
            num_categories = int(len(analyst_list)/3)
            num_analysts = np.sum(np.array(analyst_list)[[(index-1)+2*index for index in range(1, num_categories+1)]])
            
            return num_analysts
        
        try:
            current_price, low_price = get_prices(soup_tag)
            number_of_analysts = get_number_of_analysts(soup_tag)
            
            print(current_price, number_of_analysts)
            
            if current_price < low_price and number_of_analysts >= 4:
                interesting_stocks.append((symbol, ((low_price-current_price)/current_price)*100))
                
        except IndexError:
            print("No data found for symbol: " + symbol)
            
    return interesting_stocks

In [269]:
def get_interesting_stocks_zacks(interesting_stocks):
    
    updated_interesting_stocks = []
    
    for entry in interesting_stocks:
        symbol, potential_upside = entry[0], entry[1]
    
        headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.13; rv:63.0) Gecko/20100101 Firefox/63.0'}
        r = requests.get("https://www.zacks.com/stock/quote/" + symbol + "?q=aapl", headers=headers)
        html_doc = r.text
        soup = BeautifulSoup(html_doc, 'html.parser')
        soup_tag = soup.find_all("span", attrs={"class": "composite_val"})

        try:
            if len(soup_tag) == 16:
                score_list = [char for tag in soup_tag[4:7] for char in str(tag) if char.isupper()]
                
                if score_list:
                    score_list.sort()

                    if score_list[-1] <= 'B':
                        print(entry, score_list)
                        updated_interesting_stocks.append((symbol, potential_upside, score_list))
                
        except IndexError:
            print("No data found for symbol: " + symbol)
            
    return updated_interesting_stocks
            

In [1]:
interesting_stocks = get_interesting_stocks_wsj(list_stock)

In [270]:
interesting_stocks = get_interesting_stocks_zacks(interesting_stocks)

('AMX', 2185.714285714286) ['A', 'B', 'B']
('AXTA', 3.5058430717863134) ['A', 'A', 'B']
('BABA', 568.5481536616632) ['A', 'A', 'B']
('BEST', 520.3821656050957) ['A', 'A', 'A']
('CNC', 3.0600919613128266) ['A', 'A', 'B']
('CVA', 0.7919366450683905) ['B', 'B', 'B']
('DBD', 32.2556943423953) ['A', 'B', 'B']
('DCI', 2.0749665327978613) ['B', 'B', 'B']
('DOOR', 3.2565528196981686) ['A', 'A', 'B']
('EBS', 13.501385772733276) ['A', 'A', 'B']
('ELP', 13.513513513513503) ['B', 'B', 'B']
('EVR', 0.8416104329365485) ['A', 'A', 'B']
('GCO', 1.9783805833163346) ['A', 'B', 'B']
('HIG', 0.9796533534287953) ['A', 'A', 'B']
('HUM', 4.456157163392435) ['A', 'B', 'B']
('HUYA', 526.5432098765433) ['A', 'B', 'B']
('KOF', 2039.979231568017) ['A', 'B', 'B']
('LOMA', 5.348460291734199) ['A', 'A', 'A']
('LXFR', 11.867704280155648) ['A', 'A', 'B']
('NCR', 0.9336361342417295) ['A', 'A', 'B']
('NUS', 1.2609858616736656) ['A', 'A', 'B']
('PJT', 15.079958152742496) ['A', 'A', 'B']
('PRG', 29.958960328317374) ['A', 

In [273]:
interesting_stocks = sorted(interesting_stocks, key=lambda tup: tup[1])

In [274]:
interesting_stocks

[('WDR', 0.24057738572573667, ['A', 'B', 'B']),
 ('CVA', 0.7919366450683905, ['B', 'B', 'B']),
 ('EVR', 0.8416104329365485, ['A', 'A', 'B']),
 ('NCR', 0.9336361342417295, ['A', 'A', 'B']),
 ('HIG', 0.9796533534287953, ['A', 'A', 'B']),
 ('NUS', 1.2609858616736656, ['A', 'A', 'B']),
 ('TRGP', 1.7164653528289866, ['A', 'A', 'B']),
 ('GCO', 1.9783805833163346, ['A', 'B', 'B']),
 ('DCI', 2.0749665327978613, ['B', 'B', 'B']),
 ('UNF', 2.6281736647182123, ['A', 'B', 'B']),
 ('RIO', 2.6485869978456513, ['A', 'A', 'B']),
 ('CNC', 3.0600919613128266, ['A', 'A', 'B']),
 ('DOOR', 3.2565528196981686, ['A', 'A', 'B']),
 ('AXTA', 3.5058430717863134, ['A', 'A', 'B']),
 ('HUM', 4.456157163392435, ['A', 'B', 'B']),
 ('TKR', 4.594695554725436, ['B', 'B', 'B']),
 ('LOMA', 5.348460291734199, ['A', 'A', 'A']),
 ('VNT', 7.561929595827902, ['A', 'A', 'A']),
 ('TISI', 8.786610878661094, ['A', 'B', 'B']),
 ('VSTO', 9.511889862327907, ['A', 'A', 'B']),
 ('LXFR', 11.867704280155648, ['A', 'A', 'B']),
 ('EBS', 13